In [11]:
import time
import pandas as pd
from sqlalchemy import create_engine
from kafka import KafkaProducer
import mysql.connector as conn
import json


In [4]:
# my-sql db connection 
mydb = conn.connect(host='localhost',user='newuser',passwd='newpassword')
cursor = mydb.cursor()

In [5]:
# Replace 'bootstrap_servers' with your Kafka broker's address.
producer = KafkaProducer(bootstrap_servers='localhost:9092')
topic_name = 'my_topic-1'


In [8]:
cursor.execute("show databases")
cursor.fetchall()
cursor.execute("use quotes")
cursor.execute("SELECT * FROM kafka_sql_cassandra")
cursor.fetchall()

[(0,
  'Cadboury Silk',
  datetime.date(2023, 5, 12),
  100,
  datetime.datetime(2023, 7, 21, 9, 12, 53),
  datetime.datetime(2023, 7, 21, 9, 12, 53)),
 (1,
  'Nestle Kitkat',
  datetime.date(2023, 5, 21),
  50,
  datetime.datetime(2023, 7, 22, 5, 13, 14),
  datetime.datetime(2023, 7, 22, 5, 13, 14)),
 (2,
  'Snickers',
  datetime.date(2023, 7, 18),
  100,
  datetime.datetime(2023, 7, 22, 5, 13, 14),
  datetime.datetime(2023, 7, 22, 5, 13, 14)),
 (3,
  'Hershey',
  datetime.date(2023, 1, 21),
  200,
  datetime.datetime(2023, 7, 22, 5, 13, 14),
  datetime.datetime(2023, 7, 22, 5, 13, 14)),
 (4,
  'Milky Way',
  datetime.date(2023, 9, 19),
  120,
  datetime.datetime(2023, 7, 22, 5, 13, 14),
  datetime.datetime(2023, 7, 22, 5, 13, 14)),
 (5,
  'Kinder Chocolate',
  datetime.date(2023, 12, 6),
  10,
  datetime.datetime(2023, 7, 22, 5, 13, 14),
  datetime.datetime(2023, 7, 22, 5, 13, 14))]

In [14]:
import json
import datetime

json.JSONEncoder.default = lambda self,obj: (obj.isoformat() if isinstance(obj, datetime.datetime) else None)

In [18]:
# Replace 'table_name' with the name of your MySQL table.
table_name = 'kafka_sql_cassandra'

# Replace 'incremental_column' with the name of the incremental column used to track new records.
incremental_column = 'id'

last_processed_id = -1

while True:
    cursor = mydb.cursor()
    query = f"SELECT * FROM {table_name} WHERE {incremental_column} = {last_processed_id}"
    cursor.execute(query)
    records = cursor.fetchall()
    cursor.close()

    print(records)

    if not records:
        time.sleep(5)  # Wait for new records if none found
        continue

    for record in records:
        # Produce the record to the Kafka topic as a JSON string.
        producer.send(topic_name, value=json.dumps(record).encode('utf-8'))

    last_processed_id = records[-1][0]  # Assuming 'id' is the first column
    last_processed_id+= 1

    # Commit the messages to the Kafka broker after producing the batch.
    producer.flush()

    if last_processed_id >5 :
        break


[]
[]
[]
[]


KeyboardInterrupt: 

In [19]:
mydb.close()
producer.close()
